## Collaborative based filtering with SVD for Movie Recommendation

&nbsp;

* A recommender system refers to a system that is capable of predicting the future preference of a set of items for a user, and recommend the top items. One key reason why we need a recommender system in modern society is that people have too much options to use from due to the prevalence of Internet. In the past, people used to shop in a physical store, in which the items available are limited. For instance, the number of movies that can be placed in a Blockbuster store depends on the size of that store. By contrast, nowadays, the Internet allows people to access abundant resources online. Netflix, for example, has an enormous collection of movies.

&nbsp;

* In this project my major goal is to implement collaborative based filtering with **SVD( Singular Value Decomposition)**.

&nbsp;

### What is Collaborative Based filtering?

&nbsp;

![](coll.png)

&nbsp;

* Collaborative based filtering is based on users’ rating, and it will recommend us movies that we haven’t watched yet, but users similar to us have, and like. To determine whether two users are similar or not, this filter considers the movies both of them watched and how they rated them. By looking at the items in common, this type of algorithm will basically predict the rating of a movie for a user who hasn’t watched it yet, based on the similar users’ rating. 

&nbsp;

* There are two kinds of Collaborative filtering:
    * Item Based : measure the similarity between target users and other users.
    * User Based : measure the similarity between the items that target users rates/ interacts with and other items.
    
&nbsp;

* Imagine there are m users and n items, we create a matrix having dimensions m*n to denote all the past ratings of items given by user. For example m{i,j} be the rating given by user i to item j. There can be a no. Of missing cells in the matrix. Collaborative filtering involves filling those missing entries in the matrices that the user hasn’t seen or rated.

&nbsp;

![](m.png)

&nbsp;

* **User-Based Collaborative Filtering**

&nbsp;

* For User based CF we need to find similar users based on their interest. There are two approach for finding similarity i.e Pearson Correlation or Cosine Similarity. Let u{i,k} denotes the similarity between user i and user k and v{i,j} denotes the rating that user i gives to item j and v{i,j} = ? if the user hasn’t rated them. The two method can be expressed in the following manner.

&nbsp;

![](pc.png)

&nbsp;

![](cs.png)

&nbsp;

* **Item-Based Collaborative Filtering**

&nbsp;

* Instead of measuring the similarity between users, Item based CF recommends items based on their similarity with the item that the target user has rated. Like wise, the similarity can be computed with Pearson Correlation or Cosine Similarity. The major difference between them is, In item based we fill the blank vertically as oppose to the horizontal manner in the user based.

&nbsp;

* However there are major limitation with this aproach 
    * The first one would be scalability.
    * The computation grows with both the customer and the product. The worst case complexity is O(mn) with m users and n items.
    * And last would sparsity there would be a lot of empty cells within the matrix.For example if we 2000 users and 1000 movies we will be having a matrix with 2 million entries (2000*1000)
    
&nbsp;

* **So how we going to solve it.**

&nbsp;

* One of the approach would be **Low Rank Matrix Factorization** using **SVD**. The approach involves leveraging the latent factor to capture the similarity between user and items. Essentialy we wnated to turn the recomendation problem into an optimization problem. 

&nbsp;

* Given a matrix that task is to simply come up with concise vector representation of users and items such that if we want to find out the rating given by user u to item v we can get by just computing the dot product of the user vector and item vector. And we want to do it fast so we want these vectors to have few dimensions as possible which is where low rank comes into play.

&nbsp;

* One common metric is **Root Mean Square Error(RMSE)** , the lower the RMSE the better the performance. Since we do not know the rating of the unseen items, we will temporarily ignore them. Namely, we are only minimzing RMSE on the known entries in the matrix. To achive RMSE, **SVD (Singular Value Decomposition)** is adopted as shown in the below formula. 

&nbsp;

![](svd.png)

&nbsp;

* X denotes the utility matrix, and U is a left singular matrix, representing the relationship between users and latent factors. S is a diagonal matrix describing the strength of each latent factor, while V transpose is a right singular matrix, indicating the similarity between items and latent factors. Now, you might wonder what do I mean by latent factor here? It is a broad idea which describes a property or concept that a user or an item have. For instance, for music, latent factor can refer to the genre that the music belongs to. SVD decreases the dimension of the utility matrix by extracting its latent factors. Essentially, we map each user and each item into a latent space with dimension r. Therefore, it helps us better understand the relationship between users and items as they become directly comparable. 

In [1]:
#importing libraries
import pandas as pd
import numpy as np

In [2]:
# importing user data from the zip file
user_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names = user_cols, encoding = 'latin-1')

# importing movie ratings from the zip file
ratings_cols = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names = ratings_cols, encoding = 'latin-1')

# importing movies data from the zip file
movies_cols = ['movie_id','title','release_date','video_release_date','imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names = movies_cols,usecols = range(5),
                     encoding = 'latin-1')

In [3]:
# importing genre dataset 
genres_list = ['unknown','Action','Adventure','Animation','Children','Comedy','Crime',
               'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
               'Romance','Sci-Fi','Thriller','War','Western']
genre = pd.read_csv('ml-100k/u.item', sep='|',names = genres_list,usecols = range(5,24),encoding = 'latin-1')

In [4]:
# dropping redundant columns
movies.drop(['video_release_date','imdb_url'],inplace=True,axis = 1)
ratings.drop('unix_timestamp',axis = 1,inplace=True)

In [5]:
# merge all the dataset into one whole dataset
dataset = pd.merge(pd.merge(movies, ratings),users)

In [6]:
# creating the utility matrix with users as rows and movies as columns
Ratings = ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# normalise all the entries of the utility matrix and convert from a dataframe to a numpy matrix
R = Ratings.values
user_mean = np.mean(R,axis=1)
Ratings_demeaned = R - user_mean.reshape(-1,1)

In [8]:
# importing svd from sparse.linalg class of scipy package
from scipy.sparse.linalg import svds
# defining U, Sigma, V transpose
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [9]:
# diagonalising the sigma matrix
sigma = np.diag(sigma)

### Making Predictions from the decomposed matrices

In [10]:
# have added the mean back to get the original ratings back
all_predicted_ratings = np.dot(np.dot(U,sigma),Vt) + user_mean.reshape(-1,1)

In [11]:
# created the approximated utitlity matrix from the decomposed vectors to get the ratings
predictions = pd.DataFrame(all_predicted_ratings,columns=Ratings.columns)
predictions.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


In [12]:
# recommendation function to return top 20 recommended movies , 
# hasn't been rated by user along with movies already rated movies 
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print 'User {0} has already rated {1} movies.'.format(userID, user_full.shape[0])
    print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [13]:
already_rated, predictions = recommend_movies(predictions, 186, movies, ratings, 20)

User 186 has already rated 92 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [14]:
already_rated.head(20)

,user_id,movie_id,rating,title,release_date
46,186,79,5,"Fugitive, The (1993)",01-Jan-1993
45,186,1016,5,Con Air (1997),06-Jun-1997
33,186,939,5,Murder in the First (1995),01-Jan-1995
27,186,117,5,"Rock, The (1996)",07-Jun-1996
26,186,44,5,Dolores Claiborne (1994),01-Jan-1994
23,186,71,5,"Lion King, The (1994)",01-Jan-1994
39,186,226,5,Die Hard 2 (1990),01-Jan-1990
78,186,159,5,Basic Instinct (1992),01-Jan-1992
40,186,300,5,Air Force One (1997),01-Jan-1997
69,186,38,5,"Net, The (1995)",01-Jan-1995


In [15]:
predictions

,movie_id,title,release_date
416,471,Courage Under Fire (1996),08-Mar-1996
218,245,"Devil's Own, The (1997)",26-Mar-1997
805,879,"Peacemaker, The (1997)",01-Jan-1997
48,54,Outbreak (1995),01-Jan-1995
283,328,Conspiracy Theory (1997),08-Aug-1997
209,234,Jaws (1975),01-Jan-1975
531,597,Eraser (1996),21-Jun-1996
264,307,"Devil's Advocate, The (1997)",01-Jan-1997
71,82,Jurassic Park (1993),01-Jan-1993
241,273,Heat (1995),01-Jan-1995


In [16]:
# model evaluation from python package Surprise.
from surprise import SVD
from surprise import Dataset,Reader
from surprise.model_selection import cross_validate

In [17]:
# Load Reader library
reader = Reader()
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']],reader)

In [18]:
# Use the SVD algorithm.
svd = SVD()
# Compute the RMSE of the SVD algorithm.
#evaluate(svd, data, measures=['RMSE'])
cross_validate(svd, data, measures=['RMSE'],cv=5)

{u'fit_time': (6.093786954879761,
  6.17189884185791,
  6.166975021362305,
  6.3643879890441895,
  6.539247035980225),
 u'test_rmse': array([0.94432547, 0.93897129, 0.93663295, 0.93761102, 0.92826548]),
 u'test_time': (0.27909183502197266,
  0.20852208137512207,
  0.2714850902557373,
  0.257763147354126,
  0.3572499752044678)}

* We get RMSE of **0.94** which is good considering I have used a small dataset, let us now train the model and predict some ratings.

In [19]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [20]:
# predict a movie ratings for movie id 1994 by user id 186
svd.predict(186, 1994)

Prediction(uid=186, iid=1994, r_ui=None, est=3.4021935823758986, details={u'was_impossible': False})

* We get an estimated rating of 3.4 for the given movie . This prediction is possible purely from similar user who have rated the same movie.